In [1]:
from __future__ import division

import numpy as np
import scipy as sp
import scipy.sparse.linalg
from scipy.sparse import SparseEfficiencyWarning
import warnings

In [2]:
# dense cholesky
cholesky_dense = lambda x: sp.linalg.cholesky(x, lower=True)

# sparse cholesky
try:
    from sksparse.cholmod import cholesky as cholesky_sparse
except ImportError: # older versions
    from scikits.sparse.cholmod import cholesky as cholesky_sparse
    print('old scikt-sparse')

In [4]:
# configs to try: seed=12, density=0.01, mul=0.5

In [6]:
seed = 12
mul1 = 0.1 # lower values ensure positive definitness
mul2 = 0.5

F = sp.sparse.rand(1000, 5, density=0.1, format='csc', random_state=seed)
S = F.dot(F.T)
S *= mul1
S = S+S.T

with warnings.catch_warnings():
    warnings.simplefilter('ignore', category=SparseEfficiencyWarning)
    S.setdiag(1 + mul2*np.random.RandomState(seed).rand(S.shape[0])) # keep 1's on diagonal
print('density: ', S.data.size / np.prod(S.shape))
S

density:  0.049594


<1000x1000 sparse matrix of type '<class 'numpy.float64'>'
	with 49594 stored elements in Compressed Sparse Column format>

In [7]:
sp.sparse.linalg.eigsh(S, k=1, return_eigenvectors=False, which='SM')

array([0.74522417])

In [8]:
L_dense = cholesky_dense(S.A)

In [9]:
L_sparse = cholesky_sparse(S)

# check that $LL^{T} = S$

numpy/scipy

In [10]:
np.linalg.norm(S.A - L_dense.dot(L_dense.T)) / np.linalg.norm(S.A)

1.9320068704658742e-16

OK

cholmod

cholmod's representation is $LL^T = PSP^T$

In [11]:
PtL = L_sparse.apply_Pt(L_sparse.L())
np.linalg.norm(S.A - PtL.dot(PtL.T)) / np.linalg.norm(S.A)

2.324570115030523e-16

OK

# check that $LL^Tx = y$ is equiv. to $Sx = y$

using the fact:
$LL^Tx = y$ $\rightarrow$
$
\begin{cases}
  L\omega &= y \\
  L^Tx    &= \omega \\
\end{cases}
$

In [12]:
x = np.random.RandomState(seed).rand(S.shape[0],)

In [13]:
y = S.dot(x)

## scipy dense

In [14]:
w_dense = sp.linalg.solve_triangular(L_dense, y, lower=True, trans=0)

In [15]:
solx_dense = sp.linalg.solve_triangular(L_dense, w_dense, lower=True, trans=1)

In [16]:
np.linalg.norm(x - solx_dense) / np.linalg.norm(x)

1.9004056755538052e-15

OK

## cholmod

In [17]:
w_sparse = L_sparse.solve_L(L_sparse.apply_P(y))

In [18]:
solx_sparse = L_sparse.apply_Pt(L_sparse.solve_Lt(w_sparse))

In [19]:
np.linalg.norm(x - solx_sparse) / np.linalg.norm(x)

0.43370385938462436

**NOT OK**

reason: CHOLMOD solves in the form:
$LDL^{T} = S$

CHOLMOD solves in the form:
$LL^Tx = y$ $\rightarrow$
$
\begin{cases}
  L\omega &= y \\
  L^Tx    &= \omega \\
\end{cases}
$

## chomod v.0.4.3

new param `use_LDLt_decomposition` (see https://github.com/scikit-sparse/scikit-sparse/issues/43 and https://github.com/scikit-sparse/scikit-sparse/issues/9#issuecomment-326213771)

In [20]:
w_sparse = L_sparse.solve_L(L_sparse.apply_P(y), use_LDLt_decomposition=False)

In [21]:
solx_sparse = L_sparse.apply_Pt(L_sparse.solve_Lt(w_sparse, use_LDLt_decomposition=False))

In [22]:
np.linalg.norm(x - solx_sparse) / np.linalg.norm(x)

1.8111110836750037e-15

ok

also can use natural ordering

In [23]:
L_sparse_nat = cholesky_sparse(S, ordering_method='natural')

In [24]:
w_sparse_nat = L_sparse_nat.solve_L(y, use_LDLt_decomposition=False)

In [25]:
solx_sparse_nat = L_sparse_nat.solve_Lt(w_sparse_nat, use_LDLt_decomposition=False)

In [26]:
np.linalg.norm(x - solx_sparse_nat) / np.linalg.norm(x)

2.0160427115323124e-15

ok

## chomod older v.0.4.3

use in the form    
$
\begin{cases}
  LD\omega &= y \\
  L^Tx    &= \omega \\
\end{cases}
$
 or $\,$
$
\begin{cases}
  L\omega &= y \\
  DL^Tx    &= \omega \\
\end{cases}
$

In [40]:
w_sparse_old = L_sparse.solve_LD(L_sparse.apply_P(y))

In [41]:
solx_sparse_old = L_sparse.apply_Pt(L_sparse.solve_Lt(w_sparse_old))

In [42]:
np.linalg.norm(x - solx_sparse_old) / np.linalg.norm(x)

1.8512584883722028e-15

ok

In [43]:
w_sparse_old = L_sparse.solve_L(L_sparse.apply_P(y))

In [44]:
solx_sparse_old = L_sparse.apply_Pt(L_sparse.solve_DLt(w_sparse_old))

In [45]:
np.linalg.norm(x - solx_sparse_old) / np.linalg.norm(x)

1.8512584883722028e-15

ok

## scipy sparse (*replace cholmod's solve_L with scipy's spsolve_triangular*)

In [46]:
with warnings.catch_warnings():
    warnings.simplefilter('ignore', category=SparseEfficiencyWarning)
    w_scipy = sp.sparse.linalg.spsolve_triangular(L_sparse.L(), L_sparse.apply_P(y), lower=True)

In [47]:
solx_scipy = L_sparse.apply_Pt(sp.sparse.linalg.spsolve_triangular(L_sparse.L().T, w_scipy, lower=False))

In [48]:
np.linalg.norm(x - solx_scipy) / np.linalg.norm(x)

1.6355303407984224e-15

OK

## scipy vs cholmod

In [52]:
np.linalg.norm(w_sparse-w_scipy) / np.linalg.norm(w_scipy)

3.6450588665930786e-16

In [53]:
np.linalg.norm(solx_sparse-solx_scipy) / np.linalg.norm(solx_scipy)

7.389403919881755e-16

In [56]:
np.linalg.norm(w_sparse_old-w_scipy) / np.linalg.norm(w_scipy)

0.1459431697378629

In [57]:
np.linalg.norm(solx_sparse_old-solx_scipy) / np.linalg.norm(solx_scipy)

7.904698267522099e-16

In [54]:
np.linalg.norm(w_sparse_nat-w_scipy) / np.linalg.norm(w_scipy)

0.9330963700494369

In [55]:
np.linalg.norm(solx_sparse_nat-solx_scipy) / np.linalg.norm(solx_scipy)

1.3533247848572229e-15